<a href="https://colab.research.google.com/github/syouten/medium-import/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
pip install java

ERROR: Could not find a version that satisfies the requirement java (from versions: none)
ERROR: No matching distribution found for java


In [11]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [47]:
jdbc_path = "C:\\Users\\JiaHan\\AppData\\Local\\Microsoft JDBC Driver 4.2 for SQL Server\\sqljdbc_4.2\\enu\\jre8\\sqljdbc42.jar"

In [82]:

# Create Spark session
spark = SparkSession.builder \
    .appName("CSV Loader") \
    .config("spark.driver.extraClassPath",jdbc_path)\
    .config('spark.executor.extraClassPath',jdbc_path)\
    .getOrCreate()


In [83]:
file_path = "/content/homework/data/ADDRESS.csv"

In [84]:

# Load CSV data
# Replace 'path/to/your/file.csv' with your actual file path
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the loaded DataFrame
df.show()

# Optionally: Print the schema of the DataFrame
df.printSchema()


+--------------------+---------+--------------------+--------------------+----------------+------+-----------------+
|        address_guid|is_active|              line_1|              line_2|          line_3|line_4|is_person_address|
+--------------------+---------+--------------------+--------------------+----------------+------+-----------------+
|4649B5E3-4F6C-48B...|        1|21 BUKIT BATOK CR...|#23-72/73 WCEGA T...|SINGAPORE 658065|  NULL|                0|
|5B671354-1ACE-4B9...|        0|     16 JALAN KILANG|#05-01 HOI HUP BU...|SINGAPORE 159416|  NULL|                0|
|CABDCD89-5F62-42C...|        0|343 ANG MO KIO AV...|            #03-2176|SINGAPORE 560343|  NULL|                1|
|526D596F-F0DF-415...|        0|MUHLENREDDER NO.1...|             GERMANY|            NULL|  NULL|                1|
|83B7FFC1-F1D9-4C5...|        0|170 UPPER BUKIT T...|#02-10 BT TIMAH S...|SINGAPORE 588179|  NULL|                0|
|CF31328A-8004-431...|        0|     32 MAXWELL ROAD|  #03-07 WH

In [85]:
columns_list = ["line_1","line_2","line_3","line_4"]

In [86]:
for c in columns_list:
  df = df.withColumn(c,F.coalesce(c,F.lit("")))

In [65]:
df.show()

+--------------------+---------+--------------------+--------------------+----------------+------+-----------------+
|        address_guid|is_active|              line_1|              line_2|          line_3|line_4|is_person_address|
+--------------------+---------+--------------------+--------------------+----------------+------+-----------------+
|4649B5E3-4F6C-48B...|        1|21 BUKIT BATOK CR...|#23-72/73 WCEGA T...|SINGAPORE 658065|      |                0|
|5B671354-1ACE-4B9...|        0|     16 JALAN KILANG|#05-01 HOI HUP BU...|SINGAPORE 159416|      |                0|
|CABDCD89-5F62-42C...|        0|343 ANG MO KIO AV...|            #03-2176|SINGAPORE 560343|      |                1|
|526D596F-F0DF-415...|        0|MUHLENREDDER NO.1...|             GERMANY|                |      |                1|
|83B7FFC1-F1D9-4C5...|        0|170 UPPER BUKIT T...|#02-10 BT TIMAH S...|SINGAPORE 588179|      |                0|
|CF31328A-8004-431...|        0|     32 MAXWELL ROAD|  #03-07 WH

In [87]:
df = df.withColumn("full_address", F.concat_ws(" ",F.col("line_1"),F.col("line_2"),F.col("line_3"),F.col("line_4")))

In [88]:
df.withColumn("SG Address Flag",F.when(F.upper(df["full_address"]).contains("SINGAPORE"),"Y").otherwise("N")).show(truncate=False)

+------------------------------------+---------+---------------------------------+---------------------------+----------------+------+-----------------+---------------------------------------------------------------------+---------------+
|address_guid                        |is_active|line_1                           |line_2                     |line_3          |line_4|is_person_address|full_address                                                         |SG Address Flag|
+------------------------------------+---------+---------------------------------+---------------------------+----------------+------+-----------------+---------------------------------------------------------------------+---------------+
|4649B5E3-4F6C-48B6-B22B-000001B11D88|1        |21 BUKIT BATOK CRESCENT          |#23-72/73 WCEGA TOWER      |SINGAPORE 658065|      |0                |21 BUKIT BATOK CRESCENT #23-72/73 WCEGA TOWER SINGAPORE 658065       |Y              |
|5B671354-1ACE-4B93-972D-00000790EC89|0     

In [89]:
# JDBC URL
url = "jdbc:sqlserver://DESKTOP-5SH9O1E\SQLEXPRESS.database.windows.net:1433;database=TEST"

# Connection properties
properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
    "integratedSecurity": "true"
}



In [90]:
# Write the DataFrame to MS SQL
df.write.jdbc(url=url, table="your_table_name", mode="overwrite", properties=properties)

Py4JJavaError: An error occurred while calling o567.jdbc.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOp